# Proyectos bitbucket

## 📦Obtener los datos

#### Repositorios

Consultar los repositorios

In [ ]:
%pip install python-dotenv

import requests
import pandas as pd
from dotenv import load_dotenv
import os
from IPython.display import display


# Configura tus credenciales
# Cargar las variables de entorno desde el archivo .env
load_dotenv()

username = os.getenv("BITBUCKET_USERNAME")
app_password = os.getenv("BITBUCKET_APP_PASSWORD")
workspace = os.getenv("BITBUCKET_WORKSPACE")


url_repos = f"https://api.bitbucket.org/2.0/repositories/{workspace}"

response = requests.get(url_repos, auth=(username, app_password))

if response.status_code == 200:
    repositorios = response.json()
    print(f"Repositorios en {workspace}:")
    repos_data = []
    for repo in repositorios.get("values", []):
        repos_data.append(
            {
                "Nombre": repo.get("name"),
                "URL": repo.get("links", {}).get("html", {}).get("href"),
            }
        )
    repos_df = pd.DataFrame(repos_data)
    repos_df = repos_df.sort_values(
        by="Nombre"
    )  # Ordenar por nombre del repositorio en ascendente
    # print(repos_df)
else:
    print("Error al obtener repositorios:", response.status_code, response.text)

# Almacenar los repositorios en una variable para uso posterior
repositorios = [
    repo["Nombre"]
    for repo in repos_data
    if repo["Nombre"] not in ["pgp", "Pruebas_erp", "Inventario", "b2c", "efi"]
]


Listado de repositorios

In [ ]:
repos_df = pd.DataFrame(repositorios, columns=["Repositorio"])
display(repos_df)

#### Funciones

In [ ]:
# Función para obtener todos los elementos paginados
def get_all_items(url, params=None):
    items = []
    response = requests.get(url, params=params, auth=(username, app_password))
    if response.status_code != 200:
        print("Error:", response.status_code, response.text)
        return items
    data = response.json()
    items.extend(data.get("values", []))
    return items


#### Pull requests y commits

Consultar todos los estados de PR's.

Filtrar 50 registros por páginas, 3 páginas

In [ ]:
cantidad_paginas = 4
registros_por_pagina = 50  # Creo que no soporta más de 50 registros por página

# Obtener pull requests
pull_requests = []

for repo_slug in repositorios:
    # for repo_slug in ["fip", "b2b"]:
    pr_url = f"https://api.bitbucket.org/2.0/repositories/{workspace}/{repo_slug}/pullrequests"
    for page in range(1, cantidad_paginas + 1):
        pull_requests.extend(
            get_all_items(
                pr_url,
                params={"page": page, "pagelen": registros_por_pagina, "state": "ALL"},
            )
        )
        print(f"Obteniendo pull requests, repositorio: '{repo_slug}' - página: {page}")

print(f"✅Total de pull requests: {len(pull_requests)}\n")


In [ ]:
# Obtener commits
commits = []

for repo_slug in repositorios:
    # for repo_slug in ["fip", "b2b"]:
    commits_url = (
        f"https://api.bitbucket.org/2.0/repositories/{workspace}/{repo_slug}/commits"
    )
    for page in range(1, cantidad_paginas + 1):
        commits.extend(
            get_all_items(
                commits_url,
                params={"page": page, "pagelen": registros_por_pagina},
            )
        )
        print(f"Obteniendo commits, repositorio: '{repo_slug}' - página: {page}")

print(f"✅Total de commits: {len(commits)}")

### ⚠️Se van a listar los pull requests de los últimos **120 días**

In [ ]:
cantidad_dias = 120


print(f"\nTotal pull requests: {len(pull_requests)}")
fecha_limite = pd.Timestamp.now(tz="America/Bogota") - pd.Timedelta(days=cantidad_dias)
pull_requests = [
    pr for pr in pull_requests if pd.to_datetime(pr["created_on"]) >= fecha_limite
]
print(f"Registros de los últimos {cantidad_dias} días: {len(pull_requests)}\n")


print(f"\nTotal commits: {len(commits)}")
fecha_limite = pd.Timestamp.now(tz="America/Bogota") - pd.Timedelta(days=cantidad_dias)
commits = [
    commit
    for commit in commits
    if pd.to_datetime(commit["date"]) >= fecha_limite
    # and "Resolve:" in commit.get("message", "")
]
print(
    f"Registros de los últimos {cantidad_dias} días y con 'Resolve:' en el mensaje: {len(commits)}\n"
)

Limpiar **pull requests**:

- Agrega campos nuevos
- Elimina campos sin uso
- Ordena por created_on DESC

In [ ]:
for pr in pull_requests:
    if "author" in pr and "nickname" in pr["author"]:
        pr["author"] = pr["author"]["nickname"]
    if "source" in pr and "branch" in pr["source"] and "name" in pr["source"]["branch"]:
        pr["branch"] = pr["source"]["branch"]["name"]
        pr["type_branch"] = (
            pr["branch"].split("/")[0] if "/" in pr["branch"] else "unknown"
        )
    if (
        "source" in pr
        and "repository" in pr["source"]
        and "name" in pr["source"]["repository"]
    ):
        pr["repository"] = pr["source"]["repository"]["name"]
    if pr.get("merge_commit") and "hash" in pr["merge_commit"]:
        pr["merge_commit"] = pr["merge_commit"]["hash"]

    # Calculate days_open based on the PR state
    if "state" in pr and "created_on" in pr:
        created_on = pd.to_datetime(pr["created_on"])
        if pr["state"] == "OPEN":
            # For open PRs: days between created_on and now
            days_open = (
                pd.Timestamp.now(tz="America/Bogota")
                - created_on.tz_convert("America/Bogota")
            ).days
        elif pr["state"] == "MERGED" and "updated_on" in pr:
            # For merged PRs: days between created_on and updated_on
            updated_on = pd.to_datetime(pr["updated_on"])
            days_open = (
                updated_on.tz_convert("America/Bogota")
                - created_on.tz_convert("America/Bogota")
            ).days
        else:
            days_open = 0
        pr["days_open"] = days_open


df_pr = pd.DataFrame(pull_requests)

print(pull_requests[0])

# Sort by created_on in descending order to show most recent records first
df_pr = df_pr.sort_values(by="created_on", ascending=False)

# Eliminar columnas innecesarias
# df_pr = df_pr.drop(
#     columns=[
#         "type",
#         "title",
#         "description",
#         "reason",
#         "destination",
#         "summary",
#         "closed_by",
#         "links",
#         "source",
#     ]
# )

Limpiar **commits**

- Agrega campos nuevos
- Elimina campos sin uso
- Ordena por created_on DESC

In [ ]:
for cm in commits:
    # Extract author name from nested dictionary
    if "author" in cm and "user" in cm["author"] and "nickname" in cm["author"]["user"]:
        cm["author"] = cm["author"]["user"]["nickname"]
    if "repository" in cm and "name" in cm["repository"]:
        cm["repository"] = cm["repository"]["name"]


df_commits = pd.DataFrame(commits)


# Sort by date in descending order to show most recent records first
df_commits = df_commits.sort_values(by="date", ascending=False)

Imprimir **pull requests**

In [ ]:
# print(type(df_pr))
print(f"Cantidad de registros: {len(df_pr)}")
display(df_pr.head(3))
# display(df_pr)

Imprimir **commits**

In [ ]:
# print(type(df_commits))
print(f"Cantidad de registros: {len(df_commits)}")
display(df_commits.head(3))

## 📈Reportes

### 1. Actividad de commits

#### Procesar los datos

In [ ]:
# Install matplotlib if needed
%pip install matplotlib

import pandas as pd
import matplotlib.pyplot as plt

# Try to import seaborn
try:
    import seaborn as sns

    has_seaborn = True
except ImportError:
    has_seaborn = False
    print("Seaborn no instalado. Algunas visualizaciones serán simplificadas.")

# Make sure author is a string, not a dictionary
df_commits["author"] = df_commits["author"].astype(str)

# Convert date to datetime
df_commits["date"] = pd.to_datetime(df_commits["date"])

# Extract time periods
df_commits["day"] = df_commits["date"].dt.date
df_commits["week"] = df_commits["date"].dt.isocalendar().week
df_commits["month"] = df_commits["date"].dt.month
df_commits["year"] = df_commits["date"].dt.year

# Create year-month field
df_commits["year_month"] = df_commits["date"].dt.strftime("%Y-%m")

#### Análisis de actividad

### 3. Pull Requests (PR)

#### 3.1 Cantidades y tiempos

##### Procesar los datos y graficar

In [ ]:
import pandas as pd
# from datetime import datetime

# Análisis de Pull Requests
import matplotlib.pyplot as plt

# Verificar los estados disponibles en los PRs
pr_states = df_pr["state"].value_counts().reset_index()
pr_states.columns = ["Estado", "Cantidad"]

# Analizar PR por autor
pr_by_author = df_pr.groupby(["author", "state"]).size().unstack().fillna(0)
if "MERGED" in pr_by_author.columns:
    pr_by_author["total"] = pr_by_author.sum(axis=1)
    pr_by_author = pr_by_author.sort_values("total", ascending=False)
    pr_by_author = pr_by_author.drop(columns=["total"])

# Calcular tiempos promedio
df_pr["created_on"] = pd.to_datetime(df_pr["created_on"])
df_pr["updated_on"] = pd.to_datetime(df_pr["updated_on"])

# Métricas de tiempo para PRs cerrados/mergeados
merged_prs = df_pr[df_pr["state"] == "MERGED"].copy()
if not merged_prs.empty:
    merged_prs["time_to_merge_days"] = (
        merged_prs["updated_on"] - merged_prs["created_on"]
    ).dt.total_seconds() / 86400
    avg_merge_time = merged_prs["time_to_merge_days"].mean()
    median_merge_time = merged_prs["time_to_merge_days"].median()
    max_merge_time = merged_prs["time_to_merge_days"].max()

# Extraer tipos de ramas de los PRs
branch_types = df_pr["type_branch"].value_counts().reset_index()
branch_types.columns = ["Tipo", "Cantidad"]


# Visualizaciones
fig, axes = plt.subplots(2, 2, figsize=(15, 12))

# 📈 1. Estado de los PRs
axes[0, 0].bar(pr_states["Estado"], pr_states["Cantidad"], color="skyblue")
axes[0, 0].set_title("Estado de Pull Requests")
axes[0, 0].set_xlabel("Estado")
axes[0, 0].set_ylabel("Cantidad")

# 📈 2. PRs por autor
pr_by_author.plot(kind="bar", stacked=True, ax=axes[0, 1])
axes[0, 1].set_title("Pull Requests por Autor")
axes[0, 1].set_xlabel("Autor")
axes[0, 1].set_ylabel("Cantidad")
axes[0, 1].legend(title="Estado")


# 📈 3. Tiempo de resolución de PRs
if not merged_prs.empty:
    merged_prs.boxplot(column="time_to_merge_days", ax=axes[1, 0])
    axes[1, 0].set_title("Tiempo para Merge (días)")
    axes[1, 0].set_ylabel("Días")

    # Añadir una línea para el promedio
    axes[1, 0].axhline(
        y=avg_merge_time,
        color="red",
        linestyle="-",
        label=f"Promedio: {avg_merge_time:.1f} días",
    )
    axes[1, 0].legend()

# 📈 4. Tipos de ramas
axes[1, 1].pie(branch_types["Cantidad"], labels=branch_types["Tipo"], autopct="%1.1f%%")
axes[1, 1].set_title("Distribución de Tipos de Ramas")

plt.tight_layout()
plt.show()

# Métricas adicionales
print("\n=== Métricas de Pull Requests ===")
print(f"Total de PRs analizados: {len(df_pr)}")
print(f"PRs por estado: {dict(pr_states.values)}")
if not merged_prs.empty:
    print(f"\nTiempo promedio hasta merge: {avg_merge_time:.1f} días")
    print(f"Tiempo mediano hasta merge: {median_merge_time:.1f} días")
    print(f"Tiempo máximo hasta merge: {max_merge_time:.1f} días")


# -------------------------------------------------------------


# 📊 Análisis de eficiencia de PRs
if not merged_prs.empty:
    author_efficiency = (
        merged_prs.groupby("author")["time_to_merge_days"]
        .agg(["mean", "count"])
        .sort_values("mean")
    )
    author_efficiency.columns = ["Tiempo promedio (días)", "PRs mergeados"]

    # Crear tres gráficas para visualizar la eficiencia de los autores
    fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(18, 5))

    # 📈 Gráfica 1: Tiempo promedio hasta merge
    author_efficiency["Tiempo promedio (días)"].plot(
        kind="bar",
        color="skyblue",
        ax=ax1,
        title="Tiempo promedio hasta merge por autor",
    )
    ax1.set_xlabel("Autor")
    ax1.set_ylabel("Días")
    for i, v in enumerate(author_efficiency["Tiempo promedio (días)"]):
        ax1.text(i, v + 0.1, f"{v:.1f}", ha="center")

    # 📈 Gráfica 2: Cantidad de PRs mergeados
    author_efficiency["PRs mergeados"].plot(
        kind="bar",
        color="lightgreen",
        ax=ax2,
        title="Cantidad de PRs mergeados por autor",
    )
    ax2.set_xlabel("Autor")
    ax2.set_ylabel("Cantidad")
    for i, v in enumerate(author_efficiency["PRs mergeados"]):
        ax2.text(i, v + 0.1, str(v), ha="center")

    # 📈 Gráfica 3: Eficiencia general de los autores
    scatter = ax3.scatter(
        author_efficiency["Tiempo promedio (días)"],
        author_efficiency["PRs mergeados"],
        s=100,
        alpha=0.7,
        c=author_efficiency["Tiempo promedio (días)"],
        cmap="coolwarm_r",
    )

    # Añadir etiquetas a cada punto
    for i, author in enumerate(author_efficiency.index):
        ax3.annotate(
            author,
            (
                author_efficiency["Tiempo promedio (días)"][i],
                author_efficiency["PRs mergeados"][i],
            ),
            xytext=(5, 5),
            textcoords="offset points",
        )

    ax3.set_title("Eficiencia de autores")
    ax3.set_xlabel("Tiempo promedio (días)")
    ax3.set_ylabel("PRs mergeados")
    ax3.grid(True, linestyle="--", alpha=0.7)

    # Añadir colorbar
    cbar = plt.colorbar(scatter, ax=ax3)
    cbar.set_label("Tiempo promedio (días)")

    plt.tight_layout()
    plt.show()

    print("\n=== Eficiencia de autores (tiempo promedio hasta merge) ===")
    print(author_efficiency.to_string(float_format=lambda x: f"{x:.1f}"))

#### 3.2 Comentarios

##### PRs más largos

In [ ]:
# Identificar PRs con tiempos de revisión largos y comentarios numerosos
if "comment_count" in df_pr.columns:
    high_discussion = df_pr.sort_values("comment_count", ascending=False).head(5)
    print("\n=== PRs con más discusión (mayor número de comentarios) ===")
    high_discussion["formatted_date"] = high_discussion["created_on"].dt.strftime(
        "%d-%m-%Y"
    )
    print(
        pd.DataFrame(
            {
                "Título": high_discussion["title"],
                "Autor": high_discussion["author"],
                "Comentarios": high_discussion["comment_count"],
                "Estado": high_discussion["state"],
                "Fecha": high_discussion["formatted_date"],
            }
        ).to_string()
    )


##### Procesar los datos (primeras 3 gráficas)

In [ ]:
import pandas as pd
# from datetime import timedelta

import matplotlib.pyplot as plt

# Parámetro configurable: número de días a analizar
num_days_to_analyze = 30
# Calcular fecha límite
today = pd.Timestamp.now(tz="America/Bogota")
date_limit = today - pd.Timedelta(days=num_days_to_analyze)

# Asegurar que la columna de fecha está en el formato correcto
df_pr["created_on"] = pd.to_datetime(df_pr["created_on"])

# Filtrar PRs por fecha
recent_prs = df_pr[df_pr["created_on"] >= date_limit]

# Crear una lista de todas las fechas en el rango analizado
date_range = pd.date_range(start=date_limit, end=today, freq="D")
date_range = [d.date() for d in date_range]

# Contar PRs por día
prs_by_day = recent_prs.groupby(recent_prs["created_on"].dt.date).size()

# Asegurar que tenemos todas las fechas (incluso las que no tienen PRs)
prs_by_day = prs_by_day.reindex(date_range, fill_value=0)

##### Graficar

In [ ]:
import numpy as np


# 📈 3. Gráfico comparativo entre desarrolladores
dev_comparison = pd.DataFrame(
    {
        "Desarrollador": developers,
        "Total PRs": [
            len(recent_prs[recent_prs["author"] == dev]) for dev in developers
        ],
    }
).sort_values("Total PRs", ascending=False)

plt.figure(figsize=(10, 6))
bars = plt.bar(
    dev_comparison["Desarrollador"], dev_comparison["Total PRs"], color="teal"
)

# Añadir etiquetas de valor sobre cada barra
for bar in bars:
    height = bar.get_height()
    plt.text(
        bar.get_x() + bar.get_width() / 2.0,
        height + 0.1,
        f"{int(height)}",
        ha="center",
        fontsize=10,
    )

plt.title(
    f"Comparación de PRs creados por desarrollador (últimos {num_days_to_analyze} días)",
    fontsize=14,
)
plt.ylabel("Cantidad de PRs", fontsize=12)
plt.grid(axis="y", linestyle="--", alpha=0.7)
plt.tight_layout()
plt.show()


### Actividad en Ramas

In [ ]:
import pandas as pd
import seaborn as sns
import numpy as np
# from datetime import datetime, timedelta

# Análisis de Actividad en Ramas
import matplotlib.pyplot as plt

# Extraer información de ramas de los PRs
branches_info = []

# Obtener información de las ramas desde los PR
for pr in df_pr.to_dict("records"):
    # Solo procesamos si tenemos información de rama
    if "branch" in pr and "type_branch" in pr:
        # Extraer tipo de rama (feature, bugfix, etc)
        branch_type = pr["type_branch"]
        # Nombre de la rama
        branch_name = pr["branch"]
        # Repositorio
        repo = pr["repository"]
        # Fecha de creación (estimada desde la PR)
        created_date = pr["created_on"]
        # Fecha de cierre (si está cerrada)
        closed_date = pr["updated_on"] if pr["state"] != "OPEN" else None
        # Estado actual
        state = pr["state"]
        # Autor de la rama (asumimos que es el mismo que el de la PR)
        author = pr["author"]

        # Agregar a la lista
        branches_info.append(
            {
                "branch_name": branch_name,
                "branch_type": branch_type,
                "repository": repo,
                "author": author,
                "created_date": created_date,
                "closed_date": closed_date,
                "state": state,
                "days_active": (pd.Timestamp.now(tz="UTC") - created_date).days
                if state == "OPEN"
                else (closed_date - created_date).days
                if closed_date
                else None,
            }
        )

# Crear DataFrame con la información de ramas
df_branches = pd.DataFrame(branches_info)

# 1. Análisis general de ramas
branch_states = df_branches["state"].value_counts().reset_index()
branch_states.columns = ["Estado", "Cantidad"]

# 2. Análisis por tipo de rama
branch_types = df_branches["branch_type"].value_counts().reset_index()
branch_types.columns = ["Tipo", "Cantidad"]

# 3. Ramas por autor
branches_by_author = df_branches.groupby(["author", "state"]).size().unstack().fillna(0)
if "OPEN" not in branches_by_author.columns:
    branches_by_author["OPEN"] = 0
if "MERGED" not in branches_by_author.columns:
    branches_by_author["MERGED"] = 0
if "DECLINED" not in branches_by_author.columns:
    branches_by_author["DECLINED"] = 0

# 4. Tiempo activo de las ramas
branch_lifetime = (
    df_branches.dropna(subset=["days_active"])
    .groupby(["branch_type", "state"])["days_active"]
    .agg(["mean", "median", "max"])
    .reset_index()
)

# 5. Ramas abiertas vs cerradas por repositorio
branches_by_repo = (
    df_branches.groupby(["repository", "state"]).size().unstack().fillna(0)
)

# Visualizaciones
fig, axes = plt.subplots(2, 2, figsize=(16, 14))

# 1. Estado de las ramas
sns.barplot(
    x="Estado", y="Cantidad", data=branch_states, ax=axes[0, 0], palette="viridis"
)
axes[0, 0].set_title("Estado de las Ramas", fontsize=14)
for i, v in enumerate(branch_states["Cantidad"]):
    axes[0, 0].text(i, v + 0.1, str(v), ha="center")

# 2. Tipos de ramas
sns.barplot(x="Tipo", y="Cantidad", data=branch_types, ax=axes[0, 1], palette="muted")
axes[0, 1].set_title("Distribución por Tipo de Rama", fontsize=14)
for i, v in enumerate(branch_types["Cantidad"]):
    axes[0, 1].text(i, v + 0.1, str(v), ha="center")

# 3. Ramas por autor
branches_by_author.plot(kind="bar", stacked=True, ax=axes[1, 0], colormap="tab10")
axes[1, 0].set_title("Ramas por Autor y Estado", fontsize=14)
axes[1, 0].set_xlabel("Autor")
axes[1, 0].set_ylabel("Número de Ramas")
axes[1, 0].legend(title="Estado")

# 4. Tiempo promedio de las ramas abiertas
open_branches = df_branches[df_branches["state"] == "OPEN"]
if not open_branches.empty:
    sns.boxplot(x="branch_type", y="days_active", data=open_branches, ax=axes[1, 1])
    axes[1, 1].set_title("Tiempo de Ramas Abiertas por Tipo", fontsize=14)
    axes[1, 1].set_xlabel("Tipo de Rama")
    axes[1, 1].set_ylabel("Días Activa")

plt.tight_layout()
plt.show()

# 5. Gráfico adicional: Evolución de ramas a lo largo del tiempo
plt.figure(figsize=(14, 8))

# Crear fechas para el período analizado
start_date = df_branches["created_date"].min().date()
end_date = pd.Timestamp.now(tz="UTC").date()
date_range = pd.date_range(start=start_date, end=end_date, freq="D")

# Contar ramas creadas por día
df_branches["created_day"] = df_branches["created_date"].dt.date


# Imprimir estadísticas
print("\n===== ANÁLISIS DE ACTIVIDAD EN RAMAS =====")
print(f"Total de ramas analizadas: {len(df_branches)}")
print(f"Ramas por estado: {dict(branch_states.values)}")
print(f"Ramas por tipo: {dict(branch_types.values)}")
print("\nRamas abiertas por tiempo:")
if not open_branches.empty:
    open_by_time = (
        open_branches.groupby("branch_type")["days_active"]
        .agg(["count", "mean", "median", "max"])
        .reset_index()
    )
    for _, row in open_by_time.iterrows():
        print(
            f"- {row['branch_type']}: {row['count']} ramas, promedio {row['mean']:.1f} días, máximo {row['max']} días"
        )

# Detectar ramas potencialmente abandonadas (abiertas por más de 30 días)
abandoned = open_branches[open_branches["days_active"] > 30].sort_values(
    "days_active", ascending=False
)
if not abandoned.empty:
    print("\nRamas potencialmente abandonadas (>30 días abiertas):")
    for _, row in abandoned.iterrows():
        print(
            f"- {row['branch_name']} ({row['repository']}): {row['days_active']} días, autor: {row['author']}"
        )
else:
    print("\nNo hay ramas potencialmente abandonadas.")

# Analizar patrones de creación de ramas por autor
author_patterns = (
    df_branches.groupby("author")["branch_type"].value_counts().unstack().fillna(0)
)
print("\nPatrones de creación de ramas por autor:")
print(author_patterns)

### Frecuencia de Merges

In [ ]:
import pandas as pd
import seaborn as sns
import numpy as np
from matplotlib.ticker import MaxNLocator

import matplotlib.pyplot as plt

# Asegurar que las fechas están en formato datetime
df_pr["created_on"] = pd.to_datetime(df_pr["created_on"])

# Agrupar por mes/semana y estado
# Decidir si agrupar por semana o mes según la cantidad de datos
date_range = (df_pr["created_on"].max() - df_pr["created_on"].min()).days

# Si el rango de datos es mayor a 90 días, agrupar por mes, de lo contrario por semana
if date_range > 90:
    # Agrupar por mes
    df_pr["period"] = df_pr["created_on"].dt.strftime("%Y-%m")
    period_name = "Mes"
else:
    # Agrupar por semana
    df_pr["period"] = df_pr["created_on"].dt.strftime("%Y-%U")
    period_name = "Semana"

# Contar PRs por período y estado
pr_counts = df_pr.groupby(["period", "state"]).size().unstack(fill_value=0)

# Asegurar que tenemos todas las posibles columnas de estado
all_states = ["OPEN", "MERGED", "DECLINED"]
for state in all_states:
    if state not in pr_counts.columns:
        pr_counts[state] = 0

# Ordenar por período (implícitamente por fecha)
pr_counts = pr_counts.sort_index()

# Calcular el tiempo promedio hasta el cierre por período
merged_pr = df_pr[df_pr["state"] == "MERGED"].copy()
if not merged_pr.empty:
    merged_pr["period"] = merged_pr["created_on"].dt.strftime(
        "%Y-%m" if date_range > 90 else "%Y-%U"
    )
    avg_time_to_merge = merged_pr.groupby("period")["days_open"].mean()

# Crear el gráfico
fig, ax1 = plt.subplots(figsize=(14, 8))

# Paleta de colores para los estados
colors = {"OPEN": "#5DA5DA", "MERGED": "#60BD68", "DECLINED": "#F15854"}

# Crear el gráfico de barras apiladas
pr_counts[all_states].plot(
    kind="bar", stacked=True, ax=ax1, color=[colors[s] for s in all_states], width=0.7
)

# Configurar el eje primario (barras)
ax1.set_xlabel(f"{period_name}", fontsize=12)
ax1.set_ylabel("Cantidad de Pull Requests", fontsize=12)
ax1.set_title(f"Estado de Pull Requests por {period_name}", fontsize=16, pad=20)
ax1.grid(axis="y", linestyle="--", alpha=0.7)

# Rotar etiquetas del eje x para mejor visualización
plt.xticks(rotation=45, ha="right")

# Añadir leyenda para las barras
bars_legend = [plt.Rectangle((0, 0), 1, 1, color=colors[state]) for state in all_states]
ax1.legend(
    bars_legend, ["Abierto", "Fusionado", "Rechazado"], loc="upper left", title="Estado"
)

# Añadir un segundo eje para el tiempo promedio hasta merge
if not merged_pr.empty:
    ax2 = ax1.twinx()
    ax2.plot(
        range(len(pr_counts)),
        avg_time_to_merge.reindex(pr_counts.index).fillna(0),
        marker="o",
        color="#F17CB0",
        linewidth=2,
        label="Tiempo hasta merge",
    )
    ax2.set_ylabel("Días promedio hasta merge", fontsize=12, color="#F17CB0")
    ax2.tick_params(axis="y", labelcolor="#F17CB0")
    ax2.grid(False)

    # Añadir leyenda para la línea
    line_legend = plt.Line2D([0], [0], color="#F17CB0", linewidth=2, marker="o")
    ax2.legend([line_legend], ["Tiempo promedio hasta merge"], loc="upper right")

# Ajustar para que solo muestre enteros en el eje y
ax1.yaxis.set_major_locator(MaxNLocator(integer=True))

# Añadir valores sobre las barras para total por período
for i, total in enumerate(pr_counts.sum(axis=1)):
    ax1.text(i, total + 0.3, str(int(total)), ha="center", fontweight="bold")

# Añadir un cuadro de texto con guía de interpretación
plt.figtext(
    0.5,
    -0.05,
    "CÓMO INTERPRETAR ESTE GRÁFICO:\n"
    "• Las barras apiladas muestran la cantidad de PRs en cada estado por período\n"
    "• La línea rosa muestra el tiempo promedio de revisión hasta el merge\n"
    "• Un incremento en la proporción de PRs abiertos puede indicar retrasos en las revisiones\n"
    "• Un aumento en los PRs rechazados podría señalar problemas de calidad de código\n"
    "• El tiempo promedio de revisión es un indicador clave de eficiencia del equipo",
    ha="center",
    fontsize=10,
    bbox=dict(facecolor="lightyellow", alpha=0.5, boxstyle="round,pad=0.5"),
)

plt.tight_layout(rect=[0, 0.08, 1, 0.95])

# Calcular algunas métricas para mostrar en el título
total_prs = len(df_pr)
merged_pct = (df_pr["state"] == "MERGED").mean() * 100
open_pct = (df_pr["state"] == "OPEN").mean() * 100
avg_time_to_merge_total = merged_pr["days_open"].mean() if not merged_pr.empty else 0

plt.suptitle(
    f"Resumen: {total_prs} PRs totales | {merged_pct:.1f}% fusionados | {open_pct:.1f}% abiertos | "
    f"Tiempo promedio hasta merge: {avg_time_to_merge_total:.1f} días",
    fontsize=12,
    y=0.98,
)

plt.show()


# Realizar análisis de los PRs con revisiones más lentas
print("\n===== ANÁLISIS DE PRs CON REVISIONES LENTAS =====")
print(f"Se encontraron {len(slow_reviews)} PRs con tiempo de revisión mayor a 7 días")

# Calcular estadísticas por autor
author_counts = slow_reviews["author"].value_counts()
print("\nDistribución por autor:")
for author, count in author_counts.items():
    percentage = count / len(slow_reviews) * 100
    print(f"- {author}: {count} PRs lentos ({percentage:.1f}% del total)")

# Calcular estadísticas por repositorio
repo_counts = slow_reviews["repository"].value_counts()
print("\nDistribución por repositorio:")
for repo, count in repo_counts.items():
    percentage = count / len(slow_reviews) * 100
    print(f"- {repo}: {count} PRs lentos ({percentage:.1f}% del total)")

# Calcular estadísticas por tipo de rama
branch_counts = slow_reviews["type_branch"].value_counts()
print("\nDistribución por tipo de rama:")
for branch_type, count in branch_counts.items():
    percentage = count / len(slow_reviews) * 100
    print(f"- {branch_type}: {count} PRs lentos ({percentage:.1f}% del total)")

# Estadísticas de tiempo
avg_time = slow_reviews["time_to_close_days"].mean()
max_time = slow_reviews["time_to_close_days"].max()
min_time = slow_reviews["time_to_close_days"].min()

print(f"\nTiempo promedio de revisión: {avg_time:.1f} días")
print(f"Tiempo máximo de revisión: {max_time:.1f} días")
print(f"Tiempo mínimo de revisión: {min_time:.1f} días")

# Conclusiones
print("\n===== CONCLUSIONES Y RECOMENDACIONES =====")
print(
    "1. Los PRs lentos representan un punto de fricción importante en el proceso de desarrollo"
)
print(
    f"2. El repositorio {repo_counts.index[0]} tiene la mayor cantidad de PRs lentos ({repo_counts.iloc[0]} PRs)"
)
print(
    f"3. {author_counts.index[0]} es el autor con más PRs que han tenido revisiones largas"
)
print("4. Factores posibles que contribuyen a revisiones lentas:")
print("   - Complejidad del código (PRs grandes)")
print("   - Falta de revisores disponibles")
print("   - Dependencias con otros PRs o tareas")
print("   - Prioridades cambiantes en el equipo")
print("\nRecomendaciones para mejorar:")
print("1. Establecer SLAs para revisión de código (por ejemplo, máximo 3 días)")
print("2. Implementar revisiones por pares rotativas")
print("3. Dividir PRs grandes en cambios más pequeños y manejables")
print("4. Realizar sesiones de revisión de código programadas")
print("5. Monitorear y analizar tendencias de tiempo de revisión")